In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

class AnomalyDetectionModel(tf.keras.Model):
    def __init__(self, input_shape):
        super(AnomalyDetectionModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu', input_shape=input_shape)
        self.dense2 = tf.keras.layers.Dense(32, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

def train_anomaly_detection(data, labels, epochs=50, batch_size=64):
    model = AnomalyDetectionModel(input_shape=(data.shape[1],))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(data, labels, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    return model, history

def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Binarize predictions
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Plotting confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()

def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.grid()

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.grid()
    plt.show()

def load_and_preprocess_data(file_path):
    # Load the dataset
    try:
        data = pd.read_csv(file_path)
    except FileNotFoundError:
        print("Error: The specified data file was not found.")
        return None, None

    # Display the first few rows of the dataset
    print("Dataset head:")
    print(data.head())

    # Assuming the last column is the label for anomalies
    if 'anomaly' not in data.columns:
        print("Error: 'anomaly' column not found in the dataset.")
        return None, None

    # Separate features and labels
    labels = data.pop('anomaly').values  # Binary labels for anomalies
    features = data.values

    return features, labels

def main():
    # Load and preprocess the data
    features, labels = load_and_preprocess_data('deep_space_signals.csv')  # Update with your dataset path
    if features is None or labels is None:
        return

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Standardizing the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Dimensionality reduction
    pca = PCA(n_components=10)  # Adjust the number of components as needed
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    # Train the model
    model, history = train_anomaly_detection(X_train, y_train, epochs=50)

    # Plot training history
    plot_training_history(history)

    # Evaluate the model
    evaluate_model(model, X_test, y_test)

if __name__ == "__main__":
    main()